In [8]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv

# Load API key
load_dotenv()
api_key = os.getenv("ALPHA_VANTAGE_KEY")

# Alpha Vantage endpoint for daily stock prices
symbol = "PLTR"
url = "https://www.alphavantage.co/query"
params = {
    "function": "TIME_SERIES_DAILY",
    "symbol": symbol,
    "apikey": api_key
}

response = requests.get(url, params=params)
data = response.json()

# Convert to DataFrame
time_series = data.get("Time Series (Daily)", {})
df = pd.DataFrame.from_dict(time_series, orient = "index")


# Reset index, rename columns
df.reset_index(inplace=True)
df.rename(columns={"index": "date",
                   "1. open": "open",
                   "2. high": "high",
                   "3. low": "low",
                   "4. close": "close",
                   "5. volume": "volume",}, inplace=True)

# Parse dtypes
df["date"] = pd.to_datetime(df["date"])
for col in ["open", "high", "low", "close"]:
    df[col] = df[col].astype(float)
df["volume"] = df["volume"].astype(int)


# Validation
required_columns = ["date", "open", "high", "low", "close", "volume"]
missing_cols = [c for c in required_columns if c not in df.columns]
if missing_cols:
    raise ValueError(f"Missing columns: {missing_cols}")

na_counts = df.isna().sum()
print("NA counts:\n", na_counts)
print("Shape:", df.shape)

# Sort by date ascending
df = df.sort_values("date").reset_index(drop=True)
print(df)

# Save raw CSV
os.makedirs("../data/raw", exist_ok=True)
df.to_csv("../data/raw/PLTR_raw.csv", index=False)

print("Palantir stock data saved to data/raw/PLTR_raw.csv")


NA counts:
 date      0
open      0
high      0
low       0
close     0
volume    0
dtype: int64
Shape: (100, 6)
         date     open     high       low   close     volume
0  2025-04-01   83.890   85.280   81.8201   84.68   78980662
1  2025-04-02   82.400   88.405   82.3000   87.45   96563137
2  2025-04-03   81.250   85.680   81.0100   83.60   93878189
3  2025-04-04   80.070   80.980   71.9303   74.01  147323190
4  2025-04-07   66.650   81.800   66.1200   77.84  169083704
..        ...      ...      ...       ...     ...        ...
95 2025-08-18  175.270  177.900  171.3900  174.03   62656597
96 2025-08-19  171.360  172.300  156.9000  157.75  137922722
97 2025-08-20  152.300  156.460  142.3400  156.01  220336359
98 2025-08-21  157.170  157.970  153.8100  156.18   94678639
99 2025-08-22  155.315  163.200  151.7700  158.74  102099177

[100 rows x 6 columns]
Palantir stock data saved to data/raw/PLTR_raw.csv


In [1]:
pip install requests beautifulsoup4 pandas

Note: you may need to restart the kernel to use updated packages.


In [12]:
from bs4 import BeautifulSoup

# URL
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

# Fetch page
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Find table
table = soup.find("table", {"id": "constituents"})

# Extract headers
headers = [th.text.strip() for th in table.find_all("th")]

# Extract rows
rows = []
for tr in table.find_all("tr")[1:]:
    cells = [td.text.strip() for td in tr.find_all("td")]
    if cells:  
        rows.append(cells)

# Build DataFrame
df = pd.DataFrame(rows, columns=headers)

# Validation
print("Shape:", df.shape)
print("NA counts:\n", df.isna().sum())

if "CIK" in df.columns:
    df["CIK"] = pd.to_numeric(df["CIK"], errors="coerce")

# Save raw CSV
os.makedirs("../data/raw", exist_ok=True)
df.to_csv("../data/raw/sp500_companies.csv", index=False)

print("✅ Saved to data/raw/sp500_companies.csv")


Shape: (503, 8)
NA counts:
 Symbol                   0
Security                 0
GICS Sector              0
GICS Sub-Industry        0
Headquarters Location    0
Date added               0
CIK                      0
Founded                  0
dtype: int64
✅ Saved to data/raw/sp500_companies.csv


### Data Sources
1. **Alpha Vantage API**
   - URL: `https://www.alphavantage.co/query`
   - Params: 
     - `function = TIME_SERIES_DAILY_ADJUSTED`
     - `symbol = PLTR`
     - `outputsize = compact`
     - `datatype = json`
     - `apikey = <from .env>`

2. **Wikipedia (S&P 500 companies)**
   - URL: `https://en.wikipedia.org/wiki/List_of_S%26P_500_companies`
   - Method: Scrape using `requests` + `BeautifulSoup`
   - Target: Table with `id="constituents"`

---

### Validation Logic
- **Alpha Vantage**
  - Ensure `"Time Series (Daily)"` exists in JSON
  - Check required columns: `[date, open, high, low, close, adjusted_close, volume]`
  - Validate dtypes:
    - `date` → datetime
    - price fields → float
    - `volume` → int
  - Print shape + NA counts

- **Wikipedia Table**
  - Ensure table with `id="constituents"` is found
  - Validate column headers (`Symbol`, `Security`, `GICS Sector`, etc.)
  - Convert `CIK` to numeric
  - Print shape + NA counts

### Assumptions & Risks
- **API Limits**: Alpha Vantage free tier allows only 5 requests per minute. Risk of `Note` responses if limit exceeded.  
- **Data Completeness**: Alpha Vantage may not provide full history with `outputsize=compact`. Assumption: compact (100 days) is sufficient for this notebook.  
- **Data Accuracy**: Wikipedia tables may not always be up-to-date. Assumption: Wikipedia list is reasonably current but could lag official S&P sources.  
- **Schema Stability**: Both Alpha Vantage JSON keys and Wikipedia table structure are assumed to remain stable; changes may break parsing logic.  
- **Missing Data**: Assumption that occasional missing values (`NaN`) can be filled or dropped without materially impacting analysis.  